In [1]:
import pandas as pd
data = pd.read_csv('data/new_product_info.csv')

In [3]:
# Load harmful ingredients from the text file
with open("data/harmful_ingredients.txt", "r") as file:
    harmful_ingredients = [line.strip() for line in file]

# Example input ingredients
input_ingredients = ["WATER/EAU, SODIUM LAURYLSULFATE, SODIUM LAURETH SULFATE, COCAMIDOPROPYL BE- TAINE, GLYCOL DISTEARATE, DIMETHICONE, SODIUM CITRATE, COCAMIDE MEA, SODIUM XYLENESULFONATE, FRAGRANCE/ PARFUM, CITRIC ACID, SODIUM BENZOATE, POLYQUATERNI- UM-76, SODIUM CHLORIDE, TETRASODIUM EDTA, PANTHE- NOL, PANTHENYL ETHYL ETHER, METHYLCHLOROISOTHIAZOLIN- ONE, METHYLISOTHIAZOLINONE"]

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Batch process harmful ingredient embeddings
batch_size = 1000  # Adjust based on your system's memory
harmful_embeddings = []
for i in range(0, len(harmful_ingredients), batch_size):
    batch = harmful_ingredients[i:i + batch_size]
    embeddings = model.encode(batch, convert_to_tensor=True)
    harmful_embeddings.append(embeddings)
harmful_embeddings = np.vstack(harmful_embeddings)  # Combine all batches


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import faiss

# Create a FAISS index
embedding_dim = harmful_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)  # L2 (Euclidean) distance
index.add(harmful_embeddings)  # Add harmful ingredient embeddings to the index

# Encode input ingredients
input_embeddings = model.encode(input_ingredients)

# Search for the nearest neighbors
distances, indices = index.search(input_embeddings, k=5)  # Top 5 similar ingredients

# Map results back to harmful ingredients
for i, ingredient in enumerate(input_ingredients):
    print(f"Input Ingredient: {ingredient}")
    print("Potentially Harmful Matches:")
    for idx in indices[i]:
        print(f"- {harmful_ingredients[idx]} (Distance: {distances[i][idx]:.4f})")


Input Ingredient: WATER/EAU, SODIUM LAURYLSULFATE, SODIUM LAURETH SULFATE, COCAMIDOPROPYL BE- TAINE, GLYCOL DISTEARATE, DIMETHICONE, SODIUM CITRATE, COCAMIDE MEA, SODIUM XYLENESULFONATE, FRAGRANCE/ PARFUM, CITRIC ACID, SODIUM BENZOATE, POLYQUATERNI- UM-76, SODIUM CHLORIDE, TETRASODIUM EDTA, PANTHE- NOL, PANTHENYL ETHYL ETHER, METHYLCHLOROISOTHIAZOLIN- ONE, METHYLISOTHIAZOLINONE
Potentially Harmful Matches:
- Diazolidinyl urea,DMDM hydantoin,3-dimethylol-5,5-dimethylhydantoin,Imidazolidinyl urea,Sodium hydroxymethylglycinate,Quaternium-15,Dowicil 200,N-(3-chloroallyl) hexaminium chloride,p-phenylenediamine,PPD,Coal-tar,Nickel,Gold (Distance: 14.7428)
- Latex,Amyl cinnamal,Amylcinnamyl alcohol,Anisyl alcohol,Benzyl alcohol,Benzyl benzoate,Benzyl cinnamate,Benzyl salicylate,Cinnamyl alcohol,Cinnamaldehyde,Citral,Citronellol,Coumarin,Eugenol,Farnesol,Geraniol,Hexyl,cinnamaladehyde,Hydroxycitronellal,Hydroxyisohexyl 3-cyclohexene carboxaldehyde,HICC,Lyral,Isoeugenol,Lilial,d-Limonene,Linalo